# <span style="color: blue;">Part 2.1.1: Curation of model based - cofactors of glycolysis fermentation pathways</span>

- Change direction directionality for FNRR3 (Ferrodoxin reaction, should make it able to produce hydrogen). (Check that there will not be an energy producing cycle from this-what about other h2 producing reactions)
- Removing NADPH dependent alcohol dehydrogenase (ALCD2y)
- GTP/ATP dependency

In [75]:
import reframed
from reframed import pFBA
import pandas as pd
import warnings
import copy
import collections

In [76]:
model = reframed.load_cbmodel('model_cellulolyticum_H10.xml')

In [77]:
model_universe = reframed.load_cbmodel('bigg_universe.xml')

In [78]:
%store -r gene_protein_map

## Ferredoxin and NAD(P)H balancing

**Motivation**: The model is lacking some reactions described in literature that are related to cofactor balancing. This is especially related to ferredoxin reductase activity. 



### Analysis

**Reactions involving ferredoxin that we want to be in the system (based on Desvaux 2001)**
- R_POR_syn: M_coa_c + 2.0 M_fdxo_2_2_c + M_pyr_c --> M_accoa_c + M_co2_c + 2.0 M_fdxrd_c + M_h_c
- R_FNRR: M_fdxrd_c + M_h_c + M_nad_c <-> M_fdxo_2_2_c + M_nadh_c
- R_FNRR2: M_fdxrd_c + M_h_c + M_nadp_c <-> M_fdxo_2_2_c + M_nadph_c
- R_FNRR3: M_fdxo_2_2_c + M_h2_c --> M_fdxrd_c + 2.0 M_h_c


**Reactions involving ferredoxin currently in model**

In [79]:
for rxn in model.get_metabolite_reactions('M_fdxrd_c'):
    print(str(model.reactions[rxn])+ ": "+str(model.reactions[rxn].gpr))

R_CO2FO: M_co2_c + M_fdxrd_c + 2.0 M_h_c <-> M_co_c + M_fdxo_2_2_c + M_h2o_c [-inf, 0.0]: G_WP_015926513_1
R_FNRR3: M_fdxo_2_2_c + M_h2_c --> M_fdxrd_c + 2.0 M_h_c: G_WP_015925729_1
R_MECDPDH4E: M_2mecdp_c + M_fdxrd_c + M_h_c <-> M_fdxo_2_2_c + M_h2mb4p_c + M_h2o_c: G_WP_012634895_1
R_NOR_syn_1: 6.0 M_fdxrd_c + 7.0 M_h_c + M_no2_c --> 6.0 M_fdxo_2_2_c + 2.0 M_h2o_c + M_nh3_c: None
R_POR_syn: M_coa_c + 2.0 M_fdxo_2_2_c + M_pyr_c --> M_accoa_c + M_co2_c + 2.0 M_fdxrd_c + M_h_c: (G_WP_015924110_1 and G_WP_015924674_1)
R_SULR_syn: 6.0 M_fdxrd_c + 8.0 M_h_c + M_so3_c --> 6.0 M_fdxo_2_2_c + 3.0 M_h2o_c + M_h2s_c: None
R_VOR2bE: M_3mob_c + M_coa_c + M_fdxo_2_2_c --> M_co2_c + M_fdxrd_c + M_h_c + M_ibcoa_c: G_WP_015924108_1


*Of these reactions, two do not have a GPR*:

- R_NOR_syn_1: From Synechocystis sp. PCC68 model (iSynCJ816) - involved in [nitrogen metabolism and nitrate assimilation](https://www.brenda-enzymes.org/enzyme.php?ecno=1.7.7.1)
- R_SULR_syn: From Synechocystis sp. PCC68 model (iSynCJ816) - involved in [sulfate assimilation](https://www.brenda-enzymes.org/enzyme.php?ecno=1.8.7.1)

model.remove_reactions(["R_NOR_syn_1","R_SULR_syn"])

**POR_syn - GPRs**

In [80]:
model.reactions.R_POR_syn.gpr

(G_WP_015924110_1 and G_WP_015924674_1)

In [81]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015924110_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
616,B8I6P6,B8I6P6_RUMCH,Pyruvate ferredoxin/flavodoxin oxidoreductase,Ccel_0556,G_WP_015924110_1


In [82]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015924674_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
1110,B8I021,B8I021_RUMCH,Pyruvate flavodoxin/ferredoxin oxidoreductase ...,Ccel_1164,G_WP_015924674_1


**FNRR3 - GPRs in model**

<span style="color: red;">Not sure if this one makes sense. </span>

In [83]:
model.reactions.R_FNRR3.gpr

G_WP_015925729_1

In [84]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925729_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
387,B8I4Y3,B8I4Y3_RUMCH,Putative PAS/PAC sensor protein,Ccel_2300,G_WP_015925729_1


### Preparing for curation



In [85]:
rxn_add=[]
rxn_rm=[]
gprs={}

#### Reactions to add

- FNRR 
- FNRR2

*FNRR*

In [86]:
rxn_add.append(model_universe.reactions.R_FNRR)

*FNRR2*

In [87]:
rxn_add.append(model_universe.reactions.R_FNRR2)

Gene collected from UniProt (explanation in different document)

In [88]:
gene_protein_map[gene_protein_map['Gene names'].str.contains('Ccel_2546')]

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
2881,B8I6A9,B8I6A9_RUMCH,Oxidoreductase FAD/NAD(P)-binding domain protein,Ccel_2546,G_WP_015925963_1


In [89]:
prot = reframed.Protein()
prot.genes=['G_WP_015925963_1']
gpr = reframed.GPRAssociation()
gpr.proteins=[prot]
gprs['R_FNRR2']=gpr

#### Reactions to remove
- None

#### Reactions to change

##### Change directionality

- FNRR3

FNRR3 = copy.deepcopy(model.reactions.R_FNRR3)

FNRR3.reversible

FNRR3.reversible=True
FNRR3.lb=-1000

rxn_rm.append(model.reactions.R_FNRR3)
rxn_add.append(FNRR3)

### Curation and feasability test

In [90]:
rxn_add

[R_FNRR: M_fdxrd_c + M_h_c + M_nad_c <-> M_fdxo_2_2_c + M_nadh_c,
 R_FNRR2: M_fdxrd_c + M_h_c + M_nadp_c <-> M_fdxo_2_2_c + M_nadph_c]

In [91]:
rxn_rm

[]

#### Remove reactions 

In [92]:
for rxn in rxn_rm:
    print("Removing rxn: " + str(rxn))
    model.remove_reaction(rxn.id)

    env_empty = reframed.Environment.empty(model)
    objective= {rxn:0 for rxn in model.reactions}
    objective['R_ATPM']=1
    
    sol = reframed.FBA(model,objective=objective,constraints=env_empty)

    if abs(sol.fobj) <1e-6: #cplex tolerance
        print('There are NO energy producing cycles in the model')
        print("\n")
    else:
        print('There is at least one energy producing cycle in the model')
        print("\n")
        sol_pfba = pFBA(model,objective=objective,constraints=env_empty)

        print('These are the reactions that are a part of the energy producing cycle')

        for rxn,value in sol_pfba.values.items():
            if value>50.0: # What value to choose here?

                print("\t" + str(rxn)+": " + str(value))
        print("\n")

#### Add reactions 

In [93]:
for rxn in rxn_add:
    print("Adding rxn: " + str(rxn))
    model.add_reaction(rxn)
    
    env_empty = reframed.Environment.empty(model)
    objective= {rxn:0 for rxn in model.reactions}
    objective['R_ATPM']=1
    
    sol = reframed.FBA(model,objective=objective,constraints=env_empty)

    if abs(sol.fobj) <1e-6: #cplex tolerance
        print('There are NO energy producing cycles in the model')
        print("\n")
    else:
        print('There is at least one energy producing cycle in the model')
        sol_pfba = pFBA(model,objective=objective,constraints=env_empty)

        print('These are the reactions that are a part of the energy producing cycle')

        for rxn,value in sol_pfba.values.items():
            if value>50.0: # What value to choose here?

                print("\t" + str(rxn)+": " + str(value))
        print("\n")

Adding rxn: R_FNRR: M_fdxrd_c + M_h_c + M_nad_c <-> M_fdxo_2_2_c + M_nadh_c
There are NO energy producing cycles in the model


Adding rxn: R_FNRR2: M_fdxrd_c + M_h_c + M_nadp_c <-> M_fdxo_2_2_c + M_nadph_c
There are NO energy producing cycles in the model




#### Add GPR to reactions

In [94]:
for rxn_id,gpr in gprs.items():
    model.set_gpr_association(rxn_id,gpr)

### Cofactors in central carbon metabolism

- HEX1: ATP->GTP (Change)
- GALK2: ATP->GTP (Change)
- GALKr: ATP ->GTP (Change)
- PFK: ATP (protein is not functional G_WP_015926027_1) (remove)
- PFK_ppi: PPi -dependent PFK (Add)
- CD6P: same as PFK but uses wrong cofactor (remove)
- r0191: same as PFK but uses wrong cofactor (remove)
- ID6P: same as PFK but uses wrong cofactor (remove)
- R_CD6P', 'R_CT6PT', 'R_ID6P', 'R_PFK', 'R_PFK_3', 'R_UT6PT', 'R_r0191


**HEX1 - GPRs**

Most of the proteins are a part of the *ROK protein family*: [ROK family proteins are are bacterial proteins that compose of transcriptional repressors, sugar kinases, and uncharactarized ORFs.](https://www.ebi.ac.uk/interpro/entry/InterPro/IPR000600/)

One of them is a hexokinase. The evidence used for this part does not support changing GPR.  

In [95]:
model.reactions.R_HEX1.gpr

(G_WP_015924247_1 or G_WP_015925130_1 or G_WP_015925627_1 or G_WP_015926569_1 or G_WP_015926770_1)

In [96]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015924247_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
1438,B8I7S2,B8I7S2_RUMCH,ROK family protein,Ccel_0700,G_WP_015924247_1


In [97]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925130_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
497,B8I2M1,B8I2M1_RUMCH,ROK family protein,Ccel_1663,G_WP_015925130_1


In [98]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925627_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
2628,B8I4M8,B8I4M8_RUMCH,ROK family protein,Ccel_2190,G_WP_015925627_1


In [99]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015926569_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
1083,B8I0V5,B8I0V5_RUMCH,Hexokinase,Ccel_3221,G_WP_015926569_1


In [100]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015926770_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
259,B8I1T3,B8I1T3_RUMCH,ROK family protein,Ccel_3430,G_WP_015926770_1


**PFK - gprs**

There are two different proteins related to this reaction. They have different preference for ATP vs PPi.

In [101]:
model.reactions.R_PFK.gpr

(G_WP_015925658_1 or G_WP_015926027_1)

In [102]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015926027_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
822,B8I6U8,B8I6U8_RUMCH,ATP-dependent 6-phosphofructokinase (ATP-PFK) ...,pfkA Ccel_2612,G_WP_015926027_1


In [103]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925658_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
2707,B8I4R1,B8I4R1_RUMCH,Pyrophosphate--fructose 6-phosphate 1-phosphot...,pfp Ccel_2223,G_WP_015925658_1


### Preparing for curation

In [104]:
rxn_add=[]
rxn_rm=[]
gprs={}

#### Reactions to add
- PFK_ppi


*PFK_ppi*

In [105]:
stoichiometry={'M_g6p_c':-1,
              'M_ppi_c':-1,
              'M_fdp_c':1,
              'M_h_c':1,
              'M_pi_c':1}

PFK_ppi=reframed.CBReaction(reaction_id='R_PFK_ppi', 
                         name='diphosphate--fructose-6-phosphate 1-phosphotransferase',
                         reversible=True, 
                         stoichiometry=stoichiometry,
                         reaction_type=reframed.ReactionType.ENZYMATIC)

rxn_add.append(PFK_ppi)

prot = reframed.Protein()
prot.genes=['G_WP_015925658_1']
gpr = reframed.GPRAssociation()
gpr.proteins=[prot]
gprs['R_PFK_ppi']=gpr

In [106]:
rxn_add

[R_PFK_ppi: M_g6p_c + M_ppi_c <-> M_fdp_c + M_h_c + M_pi_c]

In [107]:
gprs

{'R_PFK_ppi': G_WP_015925658_1}

#### Reactions to remove
- CD6P
- r0191
- ID6P
- PFK

In [108]:
rxn_rm.append(model.reactions.R_CD6P)
rxn_rm.append(model.reactions.R_r0191)
rxn_rm.append(model.reactions.R_ID6P)
rxn_rm.append(model.reactions.R_PFK)

#### Reactions to change

##### Change GPR

##### Change Cofactor
- HEX1
- GALK2
- GALKr

*HEX1*

In [109]:
model.reactions.R_HEX1.gpr

(G_WP_015924247_1 or G_WP_015925130_1 or G_WP_015925627_1 or G_WP_015926569_1 or G_WP_015926770_1)

In [110]:
HEX1 = copy.deepcopy(model.reactions.R_HEX1)

In [111]:
HEX1.stoichiometry= collections.OrderedDict({'M_gtp_c':-1.0,'M_glc__D_c':-1.0,'M_gdp_c':1.0,'M_g6p_c': 1.0,'M_h_c': 1.0})

In [112]:
rxn_rm.append(model.reactions.R_HEX1)
rxn_add.append(HEX1)

*GALK2*

In [113]:
model.reactions.R_GALK2.gpr

G_WP_015926586_1

In [114]:
GALK2 = copy.deepcopy(model.reactions.R_GALK2)

In [115]:
GALK2.stoichiometry=collections.OrderedDict({'M_a_gal__D_c':-1,'M_gtp_c':-1,'M_gdp_c':1,'M_gal1p_c':1,'M_h_c':1})

In [116]:
rxn_rm.append(model.reactions.R_GALK2)
rxn_add.append(GALK2)

*GALKr*

In [117]:
model.reactions.R_GALKr.gpr

G_WP_015926586_1

In [118]:
GALKr = copy.deepcopy(model.reactions.R_GALKr)

In [119]:
GALKr.stoichiometry=collections.OrderedDict({'M_gal_c':-1,'M_gtp_c':-1,'M_gdp_c':1,'M_gal1p_c':1,'M_h_c':1})

In [120]:
rxn_rm.append(model.reactions.R_GALKr)
rxn_add.append(GALKr)

### Curation and feasability test

In [121]:
rxn_add

[R_PFK_ppi: M_g6p_c + M_ppi_c <-> M_fdp_c + M_h_c + M_pi_c,
 R_HEX1: M_gtp_c + M_glc__D_c --> M_gdp_c + M_g6p_c + M_h_c,
 R_GALK2: M_a_gal__D_c + M_gtp_c --> M_gdp_c + M_gal1p_c + M_h_c,
 R_GALKr: M_gal_c + M_gtp_c <-> M_gdp_c + M_gal1p_c + M_h_c]

In [122]:
rxn_rm

[R_CD6P: M_ctp_c + M_f6p_c --> M_cdp_c + M_fdp_c + M_h_c,
 R_r0191: M_f6p_c + M_utp_c --> M_fdp_c + M_h_c + M_udp_c,
 R_ID6P: M_f6p_c + M_itp_c --> M_fdp_c + M_h_c + M_idp_c,
 R_PFK: M_atp_c + M_f6p_c --> M_adp_c + M_fdp_c + M_h_c,
 R_HEX1: M_atp_c + M_glc__D_c --> M_adp_c + M_g6p_c + M_h_c,
 R_GALK2: M_a_gal__D_c + M_atp_c --> M_adp_c + M_gal1p_c + M_h_c,
 R_GALKr: M_atp_c + M_gal_c <-> M_adp_c + M_gal1p_c + M_h_c]

#### Remove reactions 

In [123]:
for rxn in rxn_rm:
    print("Removing rxn: " + str(rxn))
    model.remove_reaction(rxn.id)
    env_empty = reframed.Environment.empty(model)
    objective= {rxn:0 for rxn in model.reactions}
    objective['R_ATPM']=1
    
    sol = reframed.FBA(model,objective=objective,constraints=env_empty)

    if abs(sol.fobj) <1e-6: #cplex tolerance
        print('There are NO energy producing cycles in the model')
        print("\n")
    else:
        print('There is at least one energy producing cycle in the model')
        print("\n")
        sol_pfba = pFBA(model,objective=objective,constraints=env_empty)

        print('These are the reactions that are a part of the energy producing cycle')

        for rxn,value in sol_pfba.values.items():
            if value>0.0: # What value to choose here?

                print("\t" + str(rxn)+": " + str(value))
        print("\n")

Removing rxn: R_CD6P: M_ctp_c + M_f6p_c --> M_cdp_c + M_fdp_c + M_h_c
There are NO energy producing cycles in the model


Removing rxn: R_r0191: M_f6p_c + M_utp_c --> M_fdp_c + M_h_c + M_udp_c
There are NO energy producing cycles in the model


Removing rxn: R_ID6P: M_f6p_c + M_itp_c --> M_fdp_c + M_h_c + M_idp_c
There are NO energy producing cycles in the model


Removing rxn: R_PFK: M_atp_c + M_f6p_c --> M_adp_c + M_fdp_c + M_h_c
There are NO energy producing cycles in the model


Removing rxn: R_HEX1: M_atp_c + M_glc__D_c --> M_adp_c + M_g6p_c + M_h_c
There are NO energy producing cycles in the model


Removing rxn: R_GALK2: M_a_gal__D_c + M_atp_c --> M_adp_c + M_gal1p_c + M_h_c
There are NO energy producing cycles in the model


Removing rxn: R_GALKr: M_atp_c + M_gal_c <-> M_adp_c + M_gal1p_c + M_h_c
There are NO energy producing cycles in the model




In [124]:
for rxn in rxn_add:
    print("Adding rxn: " + str(rxn))
    model.add_reaction(rxn)
    
    env_empty = reframed.Environment.empty(model)
    objective= {rxn:0 for rxn in model.reactions}
    objective['R_ATPM']=1
    
    sol = reframed.FBA(model,objective=objective,constraints=env_empty)

    if abs(sol.fobj) <1e-6: #cplex tolerance
        print('There are NO energy producing cycles in the model')
        print("\n")
    else:
        print('There is at least one energy producing cycle in the model')
        sol_pfba = pFBA(model,objective=objective,constraints=env_empty)

        print('These are the reactions that are a part of the energy producing cycle')

        for rxn,value in sol_pfba.values.items():
            if abs(value)>0.0: # What value to choose here?

                print("\t" + str(rxn)+": " + str(value))
        print("\n")

Adding rxn: R_PFK_ppi: M_g6p_c + M_ppi_c <-> M_fdp_c + M_h_c + M_pi_c
There are NO energy producing cycles in the model


Adding rxn: R_HEX1: M_gtp_c + M_glc__D_c --> M_gdp_c + M_g6p_c + M_h_c
There are NO energy producing cycles in the model


Adding rxn: R_GALK2: M_a_gal__D_c + M_gtp_c --> M_gdp_c + M_gal1p_c + M_h_c
There are NO energy producing cycles in the model


Adding rxn: R_GALKr: M_gal_c + M_gtp_c <-> M_gdp_c + M_gal1p_c + M_h_c
There are NO energy producing cycles in the model




#### Add GPR to reactions

In [125]:
for rxn_id,gpr in gprs.items():
    model.set_gpr_association(rxn_id,gpr)

## Hydrogen production

One of the goal of this curation was to see if this will help the process of hydrogen production. Let's check if this is now possible. 

Hydrogen production is NOT feasible in the current conditions, even at 0 growth rate.

In [126]:
reframed.FVA(model,reactions=['R_EX_h2_e'])

{'R_EX_h2_e': [0.0, 0.0]}

Looking at the reactions involving M_h2_e and M_h2_c we see that there is no transport reaction for hydrogen. 

In [127]:
for rxn in model.get_metabolite_reactions('M_h2_e'):
    print(model.reactions[rxn])

R_HYDA1: M_h2_e + 2.0 M_h_c + M_mqn6_c --> 2.0 M_h_e + M_mql6_c
R_EX_h2_e: M_h2_e --> 


In [128]:
for rxn in model.get_metabolite_reactions('M_h2_c'):
    print(model.reactions[rxn])

R_FNRR3: M_fdxo_2_2_c + M_h2_c --> M_fdxrd_c + 2.0 M_h_c
R_H2ASE_syn: M_h_c + M_nadph_c <-> M_h2_c + M_nadp_c
R_HYD1pp: M_h2_c + 2.0 M_h_c + M_q8_c --> 2.0 M_h_p + M_q8h2_c
R_HYD2: M_h2_c + 2.0 M_h_c + M_mqn8_c --> 2.0 M_h_e + M_mql8_c


In the universal model there is a transport reaction for hydrogen with diffusion. We can ask project partners for this. 

In [129]:
model_universe.reactions.R_H2td

R_H2td: M_h2_c <-> M_h2_e

In [130]:
model.add_reaction(model_universe.reactions.R_H2td)

In [131]:
model.update()

In [132]:
reframed.FVA(model,reactions=['R_EX_h2_e'])

{'R_EX_h2_e': [0.0, 75.12836970474969]}

In [133]:
env_empty = reframed.Environment.empty(model)
objective= {rxn:0 for rxn in model.reactions}
objective['R_ATPM']=1

sol = reframed.FBA(model,objective=objective,constraints=env_empty)

if abs(sol.fobj) <1e-6: #cplex tolerance
    print('There are NO energy producing cycles in the model')
    print("\n")
else:
    print('There is at least one energy producing cycle in the model')
    sol_pfba = pFBA(model,objective=objective,constraints=env_empty)

    print('These are the reactions that are a part of the energy producing cycle')

    for rxn,value in sol_pfba.values.items():
        if value>50.0: # What value to choose here?

            print("\t" + str(rxn)+": " + str(value))
    print("\n")

There are NO energy producing cycles in the model




### pFBA results

In [134]:
reframed.pFBA(model,constraints={'R_EX_h2s_e':0}).show_values(pattern="R_EX",sort=True)

R_EX_glc__D_e -10
R_EX_h_e     -10
R_EX_no3_e   -7.50625
R_EX_pi_e    -0.883792
R_EX_cys__L_e -0.155023
R_EX_k_e     -0.122862
R_EX_mg2_e   -0.00546039
R_EX_fe3_e   -0.00491467
R_EX_fe2_e   -0.00422669
R_EX_ca2_e   -0.00327623
R_EX_cl_e    -0.00327623
R_EX_so4_e   -0.00273051
R_EX_cu2_e   -0.000446273
R_EX_mn2_e   -0.000434943
R_EX_ribflv_e -0.00028073
R_EX_zn2_e   -0.000214639
R_EX_cobalt2_e -6.2944e-05
R_EX_4hba_e   0.000140365
R_EX_d23hb_e  0.346936
R_EX_nh4_e    0.884451
R_EX_ac_e     10.8348
R_EX_co2_e    11.9292
R_EX_h2o_e    27.9927


### Summary

In [135]:
model.update()

**Remove genes/proteins**

In [136]:
[key for key, value in model.gene_to_reaction_lookup().items() if len(value)==0]

[]

In [137]:
model.remove_genes([key for key, value in model.gene_to_reaction_lookup().items() if len(value)==0])

In [138]:
model.update()

In [139]:
[key for key, value in model.gene_to_reaction_lookup().items() if len(value)==0]

[]

**Finish model**

In [140]:
model.id = "model_c_H10_part2_1_1"

In [141]:
reframed.save_cbmodel(model,filename="model_c_H10_part2_1_1.xml")

In [142]:
model_new = reframed.load_cbmodel('model_c_H10_part2_1_1.xml')

In [143]:
model_prev = reframed.load_cbmodel('model_cellulolyticum_H10.xml')

In [144]:
models_dict={model.id:{} for model in [model_new,model_prev]}
models_rxn_dict={model.id:{} for model in [model_new,model_prev]}
for model in [model,model_prev]:
    models_dict[model.id]['Reactions']=len(model.reactions)
    models_dict[model.id]['Metabolites']=len(model.metabolites)
    models_dict[model.id]['Genes']=len(model.genes)
    
    models_rxn_dict[model.id]['Enzymatic']=len(model.get_reactions_by_type(reframed.ReactionType.ENZYMATIC))
    models_rxn_dict[model.id]['Exchange']=len(model.get_reactions_by_type(reframed.ReactionType.EXCHANGE))
    models_rxn_dict[model.id]['Transport']=len(model.get_reactions_by_type(reframed.ReactionType.TRANSPORT))
    models_rxn_dict[model.id]['Sink']=len(model.get_reactions_by_type(reframed.ReactionType.SINK))
    models_rxn_dict[model.id]['Other']=len(model.get_reactions_by_type(reframed.ReactionType.OTHER))
    

**Overview models**

In [145]:
pd.DataFrame(models_dict)

,model_c_H10_part2_1_1,model_cellulolyticum_H10
Reactions,1811,1811
Metabolites,1250,1250
Genes,734,733


**Overview reactions in models**

In [146]:
pd.DataFrame(models_rxn_dict)

,model_c_H10_part2_1_1,model_cellulolyticum_H10
Enzymatic,880,883
Exchange,210,210
Transport,476,475
Sink,0,0
Other,245,243


**Reactions removed**

In [147]:
set(model_prev.reactions)-set(model_new.reactions)

{'R_CD6P', 'R_ID6P', 'R_PFK', 'R_r0191'}

**Reactions added**

In [148]:
set(model_new.reactions)-set(model_prev.reactions)

{'R_FNRR', 'R_FNRR2', 'R_H2td', 'R_PFK_ppi'}